Tools Generating


In [2]:
from langchain_community.tools import ArxivQueryRun, WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper,ArxivAPIWrapper

**generating arxiv tool**

In [3]:
api_wrapper_arxiv= ArxivAPIWrapper(top_k_results=2,doc_content_chars_max=500)
arxiv= ArxivQueryRun(api_wrapper=api_wrapper_arxiv,description="query arxiv papers")
print(arxiv.name)

arxiv


In [6]:
arxiv.invoke("attention all you need")

'Published: 2021-05-06\nTitle: Do You Even Need Attention? A Stack of Feed-Forward Layers Does Surprisingly Well on ImageNet\nAuthors: Luke Melas-Kyriazi\nSummary: The strong performance of vision transformers on image classification and other vision tasks is often attributed to the design of their multi-head attention layers. However, the extent to which attention is responsible for this strong performance remains unclear. In this short report, we ask: is the attention layer even necessary? Specifi'

**generating wikipedia tool**

In [7]:
api_wrapper_wiki=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=500)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper_wiki)
print(wiki.name)

wikipedia


In [ ]:
wiki.invoke("Artificial Intelligence")

#now we done with tools importing 

'Page: Artificial intelligence\nSummary: Artificial intelligence (AI) is the capability of computational systems to perform tasks typically associated with human intelligence, such as learning, reasoning, problem-solving, perception, and decision-making. It is a field of research in computer science that develops and studies methods and software that enable machines to perceive their environment and use learning and intelligence to take actions that maximize their chances of achieving defined goal'

In [9]:
from dotenv import load_dotenv
import os
load_dotenv()

os.environ["TAVILY_API_KEY"]=os.getenv("TAVILY_API_KEY")
os.environ["GROQ_API_KEY"]=os.getenv("GROQ_API_KEY")

In [10]:
from langchain_community.tools.tavily_search import TavilySearchResults
tavily=TavilySearchResults()

/tmp/ipykernel_16209/3543858584.py:2: LangChainDeprecationWarning: The class `TavilySearchResults` was deprecated in LangChain 0.3.25 and will be removed in 1.0. An updated version of the class exists in the `langchain-tavily package and should be used instead. To use it run `pip install -U `langchain-tavily` and import as `from `langchain_tavily import TavilySearch``.
  tavily=TavilySearchResults()


In [11]:
tavily.invoke("Latest advancements in AI research")

[{'title': "Google's year in review: 8 areas with research breakthroughs in 2025",
  'url': 'https://blog.google/innovation-and-ai/products/2025-research-breakthroughs/',
  'content': 'In 2025, Google made significant AI research breakthroughs with models like Gemini 3 and Gemma 3. These advancements improved AI\'s reasoning, multimodality, and efficiency, leading to new products and features across Google\'s portfolio. Expect more AI-driven innovations in science, computing, and tools for global challenges as Google prioritizes responsible AI development and collaboration.\n\n## Bullet points [...] ## Bullet points\n\n "Google\'s year in review: 8 areas with research breakthroughs in 2025" highlights AI advancements and more.\n Gemini 3 models showed big leaps in reasoning, multimodality, efficiency, and creative abilities.\n AI is transforming Google\'s products, from Pixel 10 to Search, with agentic capabilities.\n AI is boosting science, from genomics and healthcare to math, coding

combining all tools into single variable 

In [13]:
tools=[arxiv,wiki,tavily]

intilize the llm model


In [20]:
from langchain_groq import ChatGroq

llm=ChatGroq(model="llama-3.1-8b-instant",temperature=0)

In [21]:
llm.invoke("what is ai")


AIMessage(content='AI, or Artificial Intelligence, refers to the simulation of human intelligence in machines that are programmed to think and learn like humans. The term can also be applied to any machine that exhibits traits associated with a human mind such as learning and problem-solving.\n\nAI technology is based on the principle of creating algorithms that can process data, identify patterns, and make decisions with minimal human intervention. This is achieved through various techniques, including:\n\n1. **Machine Learning (ML)**: A subset of AI that involves training algorithms on data to enable them to learn and improve their performance over time.\n2. **Deep Learning (DL)**: A type of ML that uses neural networks with multiple layers to analyze and interpret data.\n3. **Natural Language Processing (NLP)**: A field of AI that deals with the interaction between computers and humans in natural language.\n4. **Computer Vision**: A field of AI that enables computers to interpret an

In [23]:
#connect the tools with     the llm 
llm_with_tools=llm.bind_tools(tools)

In [27]:
llm_with_tools.invoke("what is latest news on ai")

AIMessage(content='', additional_kwargs={'tool_calls': [{'id': '855btk166', 'function': {'arguments': '{"query":"latest news on AI"}', 'name': 'tavily_search_results_json'}, 'type': 'function'}]}, response_metadata={'token_usage': {'completion_tokens': 21, 'prompt_tokens': 459, 'total_tokens': 480, 'completion_time': 0.03813202, 'completion_tokens_details': None, 'prompt_time': 0.093212513, 'prompt_tokens_details': None, 'queue_time': 0.055990488, 'total_time': 0.131344533}, 'model_name': 'llama-3.1-8b-instant', 'system_fingerprint': 'fp_ff2b098aaf', 'service_tier': 'on_demand', 'finish_reason': 'tool_calls', 'logprobs': None, 'model_provider': 'groq'}, id='lc_run--019bfa05-3f06-7102-9ea5-4643baaa8df6-0', tool_calls=[{'name': 'tavily_search_results_json', 'args': {'query': 'latest news on AI'}, 'id': '855btk166', 'type': 'tool_call'}], invalid_tool_calls=[], usage_metadata={'input_tokens': 459, 'output_tokens': 21, 'total_tokens': 480})

** now we build our agentic application**

In [2]:
from typing_extensions import TypedDict
from langchain_core.messages import AnyMessage
from typing import Annotated
from langgraph.graph.message import add_messages

In [3]:
class state(TypedDict):
    messages:Annotated[list[AnyMessage],add_messages]
    

In [5]:
from IPython.display import display,Image
from langgraph.graph import StateGraph,START,END
from langgraph.prebuilt import ToolNode
from langgraph.prebuilt import tools_condition